# 2021 Championship EDA

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, RidgeCV, Lasso, \
LassoCV, ElasticNet, ElasticNetCV
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.decomposition import PCA

In [2]:
team_stats = pd.read_csv('../data/FS_champ_team_2021.csv')
games = pd.read_csv('../data/FS_champ_games_2021.csv')
goal_timings = pd.read_csv('../data/FS_Champ2021_Goal_timings.csv')

In [3]:
team_stats.describe()

,matches_played,matches_played_home,matches_played_away,suspended_matches,wins,wins_home,wins_away,draws,draws_home,draws_away,...,goals_conceded_min_61_to_70,goals_conceded_min_71_to_80,goals_conceded_min_81_to_90,draw_percentage_overall,draw_percentage_home,draw_percentage_away,loss_percentage_ovearll,loss_percentage_home,loss_percentage_away,over145_corners_percentage
count,24.000000,24.000000,24.000000,24.0,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,...,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000
mean,46.416667,23.208333,23.208333,0.0,17.375000,9.583333,7.791667,11.666667,5.833333,5.833333,...,6.625000,5.541667,9.208333,25.166667,25.000000,25.125000,37.458333,33.708333,41.333333,4.583333
std,0.974308,0.508977,0.508977,0.0,5.427887,3.399702,2.918891,2.632021,2.119612,2.258639,...,2.533472,2.395270,3.833255,5.654292,8.968544,9.728097,11.754663,14.131307,14.187155,3.295803
min,46.000000,23.000000,23.000000,0.0,11.000000,5.000000,4.000000,6.000000,2.000000,2.000000,...,3.000000,1.000000,4.000000,13.000000,9.000000,9.000000,15.000000,8.000000,17.000000,0.000000
25%,46.000000,23.000000,23.000000,0.0,12.750000,7.000000,6.000000,10.000000,4.000000,4.000000,...,4.750000,4.000000,6.000000,22.000000,17.000000,17.000000,30.000000,26.000000,30.000000,2.000000
50%,46.000000,23.000000,23.000000,0.0,16.000000,8.500000,7.500000,11.000000,6.000000,6.000000,...,6.500000,5.000000,9.000000,24.000000,25.500000,25.500000,39.000000,32.500000,38.500000,4.000000
75%,46.000000,23.000000,23.000000,0.0,20.000000,12.000000,9.250000,13.000000,7.000000,8.000000,...,8.250000,7.250000,11.000000,28.000000,30.000000,33.500000,46.000000,40.000000,52.000000,6.250000
max,49.000000,25.000000,25.000000,0.0,29.000000,19.000000,15.000000,17.000000,10.000000,10.000000,...,12.000000,10.000000,17.000000,37.000000,43.000000,43.000000,57.000000,61.000000,70.000000,13.000000


In [4]:
games.columns

Index(['timestamp', 'date_GMT', 'status', 'attendance', 'home_team_name',
       'away_team_name', 'referee', 'Game Week', 'Pre-Match PPG (Home)',
       'Pre-Match PPG (Away)', 'home_ppg', 'away_ppg', 'home_team_goal_count',
       'away_team_goal_count', 'total_goal_count', 'total_goals_at_half_time',
       'home_team_goal_count_half_time', 'away_team_goal_count_half_time',
       'home_team_goal_timings', 'away_team_goal_timings',
       'home_team_corner_count', 'away_team_corner_count',
       'home_team_yellow_cards', 'home_team_red_cards',
       'away_team_yellow_cards', 'away_team_red_cards',
       'home_team_first_half_cards', 'home_team_second_half_cards',
       'away_team_first_half_cards', 'away_team_second_half_cards',
       'home_team_shots', 'away_team_shots', 'home_team_shots_on_target',
       'away_team_shots_on_target', 'home_team_shots_off_target',
       'away_team_shots_off_target', 'home_team_fouls', 'away_team_fouls',
       'home_team_possession', 'away_te

In [5]:
games['home_team_goal_timings'].dtype

dtype('O')

# Goal Analysis

In [6]:
season_goal_total = games['total_goal_count'].sum()
season_1h_goal_total = games['total_goals_at_half_time'].sum()
percent_goals_1h = ((season_1h_goal_total)/(season_goal_total)) * 100

In [7]:
print(f'There were {season_goal_total} goals scored this season')

There were 1284 goals scored this season


In [8]:
print(f'There were {season_1h_goal_total} goals scored in the 1H this season, ')

There were 575 goals scored in the 1H this season, 


In [9]:
season_1h_goal_total/season_goal_total
#% of first half goals

0.44781931464174457

# When were these goals scored?

In [10]:
goal_timings = goal_timings.rename(columns={'home_team_goal_timings':'Goal_1', 'Unnamed: 1': 'Goal_2', 'Unnamed: 2': 'Goal_3', 
                             'Unnamed: 3': 'Goal_4', 'Unnamed: 4': 'Goal_5',
                             'Unnamed: 5': 'Goal_6', 'Unnamed: 6': 'Goal_7', 'away_team_goal_timings':'A_Goal_1', 'Unnamed: 8': 'A_Goal_2', 'Unnamed: 9': 'A_Goal_3', 
                             'Unnamed: 10': 'A_Goal_4', 'Unnamed: 11': 'A_Goal_5'})

In [11]:
len(goal_timings['Goal_1'])

557

In [12]:
# df = goal_timings['Goal_1'].append(goal_timings['Goal_2'])

# In how many games did a team have more xG but lose?